## ResNet34 - Class Activation Map (CAM)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *

In [ ]:
PATH = "~/Project_SEM/Project_Task/data/"
sz = 224
arch = resnet34
bs = 8

In [ ]:
m = arch(True)

In [ ]:
m

In [ ]:
m = nn.Sequential(*children(m)[:-2], 
                  nn.Conv2d(512, 5, 3, padding=1), 
                  nn.AdaptiveAvgPool2d(1), Flatten(), 
                  nn.LogSoftmax())

In [ ]:
m

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers = 0)

In [ ]:
learn = ConvLearner.from_model_data(m, data)

In [ ]:
learn.freeze_to(-4)

In [ ]:
m[-1].trainable

In [ ]:
m[-4].trainable

In [ ]:
lrf=learn.lr_find(end_lr=10)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot(n_skip=5, n_skip_end=1)

In [ ]:
learn.fit(1e-3, 5)

In [ ]:
learn.fit(1e-3, 5, cycle_len=1)

In [ ]:
learn.unfreeze()
#lrs=np.array([5e-5,5e-4,5e-3])

In [ ]:
lrf=learn.lr_find(end_lr=10)

In [ ]:
learn.sched.plot(n_skip=5, n_skip_end=1)

In [ ]:
learn.fit(1e-4, 3, cycle_len=3, cycle_mult=2)

## Class Activation Map (CAM)

In [ ]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [ ]:
lst = iter(data.val_dl)
x,y = next(lst)

In [ ]:
x,y = next(lst)

In [ ]:
x,y = x[None,1], y[None,1]
vx = Variable(x.cuda(), requires_grad=True)

In [ ]:
dx = data.val_ds.denorm(x)[0]
plt.imshow(dx);

In [ ]:
sfs = [SaveFeatures(o) for o in [m[-7], m[-6], m[-5], m[-4]]]

In [ ]:
%time py = m(Variable(x.cuda()))

In [ ]:
for o in sfs: o.remove()

In [ ]:
[o.features.size() for o in sfs]

In [ ]:
py = np.exp(to_np(py)[0]); py

In [ ]:
data.classes

In [ ]:
feat = np.maximum(0,to_np(sfs[3].features[0]))
feat.shape

In [ ]:
f2=np.dot(np.rollaxis(feat,0,3), py)
f2-=f2.min()
f2/=f2.max()
f2

In [ ]:
plt.imshow(dx)
plt.imshow(scipy.misc.imresize(f2, dx.shape), alpha=0.5, cmap='hot');

## Model

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

accuracy_np(probs,y)

In [ ]:
preds = np.argmax(probs, axis=1)
#probs = probs[:,1]

from sklearn.metrics import confusion_matrix
from fastai.plots import *
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)